In [3]:

import dspy
import os
from pathlib import Path
from dotenv import load_dotenv
load_dotenv(Path(r"utils/.env"))

True

In [ ]:
from utils.rec_system import RecSysDataset, RecommendationSystem
from utils.retriever import ColbertRetriever
from utils.metrics import precision_at_k
import json

#Complete list of products to retrieve from
with open(Path(r"./products.json"), 'r') as f:
    product_json = json.loads(f.read())

#Input
query = "2 in pvc pipe"
k= 5

#Model initializations
rm = ColbertRetriever(model_name = "colbert", documents = product_json["products"])
lm = dspy.LM(model = os.getenv("BASE_LLM"), api_key = os.getenv("GROQ_API_KEY"), temperature = 0.7)
dspy.settings.configure(lm = lm, rm = rm)


recsys = RecommendationSystem(k=k)
recommended_products = recsys(product_query = query)

#Dataset
dataset = RecSysDataset("recsys_dataset.json")

In [ ]:
bs_fewshot_rs = dspy.BootstrapFewShotWithRandomSearch(
    metric = precision_at_k,
    max_bootstrapped_demos=2,
    max_labeled_demos=3,
    max_rounds=1,
    num_candidate_programs=0,
    num_threads=1,
    max_errors=10,
    stop_at_score=None,
    metric_threshold=None,
)

In [ ]:
new_program = bs_fewshot_rs.compile(
    student = recsys,
    trainset = dataset.train(0,6),
    valset = dataset.dev(6,8)
)